## Importing Libraries

In [1]:
import pandas as pd
import numpy as np

## Defining the dataset

**Dataframe columns:**
1. zone: The zone number 
2. curr_air_flow: The current air flow in the zone (in CFM)
3. air_flow_thr: The threshold value of air flow, beyond which the zone is deemed 'large flow zone'
4. curr_temp: The current temperature in the zone (in degree F)
5. stp: The set point for the given zone (in degree F)
6. air_flow_dev: The deviation of current air flow from the threshold value (curr_air_flow - air_flow_thr)
7. temp_dev: The deviation of current temperature from the set point temperature (curr_temp - stp)

In [156]:
df = pd.DataFrame({'zone':np.arange(1,16),
                   'curr_air_flow':np.random.randint(0,2000,15),
                   'air_flow_thr':15*[1000],
                   'curr_temp':np.random.uniform(65,80,15).round(2),
                   })
df['stp'] = df['curr_temp'] + np.random.uniform(-5,5,15).round(2)
df['air_flow_dev'] = df['curr_air_flow'] - df['air_flow_thr']
df['temp_dev'] = df['curr_temp'] - df['stp']
df

,zone,curr_air_flow,air_flow_thr,curr_temp,stp,air_flow_dev,temp_dev
0,1,1747,1000,72.09,75.60,747,-3.51
1,2,599,1000,69.08,72.26,-401,-3.18
2,3,1637,1000,77.55,73.64,637,3.91
3,4,1101,1000,79.33,83.11,101,-3.78
4,5,1578,1000,66.12,69.26,578,-3.14
5,6,192,1000,71.02,66.66,-808,4.36
6,7,1427,1000,79.89,79.90,427,-0.01
7,8,1374,1000,78.37,78.17,374,0.20
8,9,511,1000,77.50,81.33,-489,-3.83
9,10,795,1000,67.76,65.18,-205,2.58


## Data Preprocessing

**New Columns:**

1. air_flow_ratio: Ratio of the current air flow to the threshold air flow
2. temp_ratio: Ratio of the current temperature to the set point temperature
3. rel_air_flow_dev: Deviation of the current air flow from the threshold value as a fraction of the threshold value
4. rel_temp_dev: Deviation of the current temperature from the set point as a fraction of the set point

In [157]:
df['air_flow_ratio'] = (df['curr_air_flow']/df['air_flow_thr']).round(3)
df['temp_ratio'] = (df['curr_temp']/df['stp']).round(3)
df['rel_air_flow_dev'] = (df['air_flow_dev']/df['air_flow_thr']).round(3)
df['rel_temp_dev'] = (df['temp_dev']/df['stp']).round(3)
df

,zone,curr_air_flow,air_flow_thr,curr_temp,stp,air_flow_dev,temp_dev,air_flow_ratio,temp_ratio,rel_air_flow_dev,rel_temp_dev
0,1,1747,1000,72.09,75.60,747,-3.51,1.747,0.954,0.747,-0.046
1,2,599,1000,69.08,72.26,-401,-3.18,0.599,0.956,-0.401,-0.044
2,3,1637,1000,77.55,73.64,637,3.91,1.637,1.053,0.637,0.053
3,4,1101,1000,79.33,83.11,101,-3.78,1.101,0.955,0.101,-0.045
4,5,1578,1000,66.12,69.26,578,-3.14,1.578,0.955,0.578,-0.045
5,6,192,1000,71.02,66.66,-808,4.36,0.192,1.065,-0.808,0.065
6,7,1427,1000,79.89,79.90,427,-0.01,1.427,1.000,0.427,-0.000
7,8,1374,1000,78.37,78.17,374,0.20,1.374,1.003,0.374,0.003
8,9,511,1000,77.50,81.33,-489,-3.83,0.511,0.953,-0.489,-0.047
9,10,795,1000,67.76,65.18,-205,2.58,0.795,1.040,-0.205,0.040


## Scoring Method

The idea is to identify zones having large air flow (ie positive deviation from threshold) and low temperature (ie, negative deviation from set point) and prioritize them for AC shutdown.

To do this, we can construct a score which prioritizes such zones. For example: Ratio of change in temperature to change in air flow.

For large zones, the air_flow_ratio would be > 1 \
For small zones, the air_flow_ratio would be < 1 \
For low temperature zones, the temp_ratio would be < 1\
For high temperature zones, the temp_ratio would be > 1

**zone score** = temp_ratio / air_flow_ratio

Therefore, \
a score < 1 implies that either the zone is large or the temperautre is low or both \
a score > 1 implies that either the zone is small or the temperature is high or both

In [158]:
ndf = df[['zone','curr_air_flow','air_flow_thr','curr_temp','stp','air_flow_dev','temp_dev']]
ndf['score1'] = (df['temp_ratio']/df['air_flow_ratio']).round(3)
ndf = ndf.sort_values(['score1'], ascending=True).reset_index(drop=True)
ndf['ac_shutdown_priority'] = np.arange(1,16)
ndf

/Users/anirudhchandra/anaconda3/envs/machine_learning/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,zone,curr_air_flow,air_flow_thr,curr_temp,stp,air_flow_dev,temp_dev,score1,ac_shutdown_priority
0,12,1938,1000,78.06,78.10,938,-0.04,0.515,1
1,1,1747,1000,72.09,75.60,747,-3.51,0.546,2
2,15,1672,1000,79.95,80.17,672,-0.22,0.596,3
3,5,1578,1000,66.12,69.26,578,-3.14,0.605,4
4,13,1655,1000,70.01,68.10,655,1.91,0.621,5
5,3,1637,1000,77.55,73.64,637,3.91,0.643,6
6,7,1427,1000,79.89,79.90,427,-0.01,0.701,7
7,8,1374,1000,78.37,78.17,374,0.20,0.730,8
8,14,1356,1000,73.10,70.06,356,3.04,0.769,9
9,4,1101,1000,79.33,83.11,101,-3.78,0.867,10
